<a href="https://colab.research.google.com/github/simulate111/Introduction-to-Human-Language-Technology/blob/main/Excercise%20task%204%3A%20creating%20a%20dataset%20from%20corpus%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import zipfile
import pandas as pd
from datasets import Dataset, DatasetDict

In [2]:
positive_url = "https://github.com/simulate111/Introduction-to-Human-Language-Technology/raw/main/imdb-positives.zip"
negative_url = "https://github.com/simulate111/Introduction-to-Human-Language-Technology/raw/main/imdb-negatives.zip"

# Acquire positive and negative files
positive_response = requests.get(positive_url)
with open("imdb-positives.zip", "wb") as f:
    f.write(positive_response.content)
with zipfile.ZipFile("imdb-positives.zip", "r") as zip_ref:
    zip_ref.extractall()

negative_response = requests.get(negative_url)
with open("imdb-negatives.zip", "wb") as f:
    f.write(negative_response.content)
with zipfile.ZipFile("imdb-negatives.zip", "r") as zip_ref:
    zip_ref.extractall()

# Read lines and labeling
with open("imdb-positives.txt", "r") as f:
    positive_lines = f.readlines()
positive_df = pd.DataFrame({'text': positive_lines, 'label': 'positive'})
display(positive_df.head())
print(positive_df.shape)
with open("imdb-negatives.txt", "r") as f:
    negative_lines = f.readlines()
negative_df = pd.DataFrame({'text': negative_lines, 'label': 'negative'})
display(negative_df.head())
print(negative_df.shape)

,text,label
0,Zentropa has much in common with The Third Man...,positive
1,Zentropa is the most original movie I've seen ...,positive
2,Lars Von Trier is never backward in trying out...,positive
3,*Contains spoilers due to me having to describ...,positive
4,That was the first thing that sprang to mind a...,positive


(25000, 2)


,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,negative
1,"""I Am Curious: Yellow"" is a risible and preten...",negative
2,If only to avoid making this type of film in t...,negative
3,This film was probably inspired by Godard's Ma...,negative
4,"Oh, brother...after hearing about this ridicul...",negative


(25000, 2)


In [3]:
# Combine both opinions
dataset_PN = pd.concat([positive_df, negative_df], ignore_index=True)
#display(dataset_PN.head())
#display(dataset_PN.tail())
#dataset_PN.shape

In [4]:
#The dataset is split into three parts: 'train', 'validation', and 'test' with relative sizes
# 80%, 10%, and 10% (respectively). Each of these parts has a roughly 50/50 distribution of 'positive' and 'negative' examples.
from sklearn.model_selection import train_test_split
train_df, val_test_df = train_test_split(dataset_PN, test_size=0.2, stratify=dataset_PN['label'])
val_df, test_df = train_test_split(val_test_df, test_size=0.5, stratify=val_test_df['label'])

In [5]:
!pip install datasets


In [6]:
#The dataset is represented using the DatasetDict and Dataset classes from the datasets library (see below)
# so that there is a top-level DatasetDict object with the keys 'train', 'validation' and 'test'
# where the values are three Dataset objects with the relavant data. (If you already did exercise 3, this organization should be familiar to you.)
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Create DatasetDict
dataset_dict = DatasetDict({'train': train_dataset, 'validation': val_dataset, 'test': test_dataset})

# Print dataset sizes
print("Train dataset size:", len(dataset_dict['train']))
print("Validation dataset size:", len(dataset_dict['validation']))
print("Test dataset size:", len(dataset_dict['test']))


Train dataset size: 40000
Validation dataset size: 5000
Test dataset size: 5000


In [7]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 40000
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 5000
    })
})

In [8]:
dataset_dict['train']

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 40000
})

In [9]:
dataset_dict['train']['text'][:2]

["This ranks as my favorite movie of all time. It's the best spoof of a science fiction movie ever; the fact that it was a sendup of Star Wars just made it all the better.<br /><br />I love slapstick. Think of this as the Marx brothers or the Three Stooges meet Star Wars. The writing is hilarious. The effects are a hoot. The free association that goes on guarantees all sorts of things coming out of left field. (I almost wet my pants when the Wookie Monster accosted the Princess.)<br /><br />Space Balls was a much longer movie, but only had about 15 minutes of good material in it, and I felt sort of ripped off afterwards, like buying a burger that turned out to be mostly filler. Hardware Wars, despite being only about 15 minutes long, would be worth paying a feature price, IMO.\n",
 "When HEY ARNOLD! first came on the air in 1996, I watched it. It was one of my favorite shows. Then the same episodes started getting shown over and over again so I got tired of waiting for new episodes and

In [10]:
set(dataset_dict['train']['label'])

{'negative', 'positive'}

In [11]:
print("Length of train' dataset:", sum(1 for item in dataset_dict['train']))
print("Positive labels in 'train' dataset:", sum(1 for pos in dataset_dict['train']['label'] if pos == 'positive'))
print("Negative labels in 'train' dataset:", sum(1 for neg in dataset_dict['train']['label'] if neg == 'negative'))

Length of train' dataset: 40000
Positive labels in 'train' dataset: 20000
Negative labels in 'train' dataset: 20000


In [12]:
print("Length of 'validation' dataset:", len(dataset_dict['validation']))
print("Positive labels in 'validation' dataset:", sum(1 for pos in dataset_dict['validation']['label'] if pos == 'positive'))
print("Negative labels in 'validation' dataset:", sum(1 for neg in dataset_dict['validation']['label'] if neg == 'negative'))

Length of 'validation' dataset: 5000
Positive labels in 'validation' dataset: 2500
Negative labels in 'validation' dataset: 2500


In [13]:
print("Length of 'test' dataset:", len(dataset_dict['test']))
print("Positive labels in 'test' dataset:", sum(1 for pos in dataset_dict['test']['label'] if pos == 'positive'))
print("Negative labels in 'test' dataset:", sum(1 for neg in dataset_dict['test']['label'] if neg == 'negative'))

Length of 'test' dataset: 5000
Positive labels in 'test' dataset: 2500
Negative labels in 'test' dataset: 2500
